# Demo on Running Acceptance Tests using Behavior-Driven Development

## Overview

In this notebook, you will generate feature files used to execute acceptance tests for an object-sorting task in the Isaac Sim simulator. The computational load for rendering the simulation is handled by a dedicated server, and the rendered environment is visualized on the VNC desktop via a WebRTC client application. Through this notebook, users can interact with the server to execute and visualize the tests.

This demo involves opening the desktop environment as the first step, followed by generating the feature file. Next, the user registers on the server, uploads the feature file, and interacts with the rest of the API endpoints to control and analyze the tests.

Once the start command is executed, please wait for approximately 20 seconds before launching the WebRTC client application. If it does not load, click `View` followed by `Reload`, and enter the assigned IP address in the `Server:` field.

### Open VNC Desktop

In [ ]:
# before executing this cell, make sure the **Simple** button at the bottom left of the window is unchecked

from utils import display_desktop
display_desktop()

## Steps to Generate Feature Files in the Remote Desktop

**Note**: The following steps are relevant only when using a terminal from the remote desktop. 
- Open a terminal, either in the desktop environment or in a separate tab within JupyterLab
- Deactivate the conda environment: `conda deactivate`
- Activate the Python virtual environment, where relevant packages are already installed: `source ~/venv/bin/activate`
- Navigate to the *bdd-dsl/examples* path: `cd ~/behave-isaac-bdd/bdd-dsl/examples/`
- Execute *generate_bdd_specs.py*: `python3 generate_bdd_specs.py`

If the feature files are generated from the terminal, skip the next step and continue from setting server IP cell

**Generate the feature file**

In [ ]:
%%bash --bg
python3 ~/behave-isaac-bdd/bdd-dsl/examples/generate_bdd_specs.py

### API endpoints
- `register_user`: Assigns the user to a GPU pod and returns its IP address (used in WebRTC client)
- `status`: Retrieves the current status of the assigned pod
- `upload`: Uploads a *.feature* file for simulation
- `start`: Starts the simulation on the assigned pod
- `stop`: Stops the simulation
- `logs?n`: Returns the last N lines from the simulation logs
- `logs_stream`: Streams live logs from the simulation
- `release`: Deregisters the user and frees the assigned pod

**Notes:**
- The API calls can be executed in any sequence that fits your custom logic
- `release` should be called at the end of the session to deregister from assigned pod
- Each user is assigned 30 minutes of usage time. Once the time expires, the user can register again to continue using the virtual lab

### import libraries and set server IP

In [ ]:
import subprocess

# ip address of server used to access its API endpoint
ws_ip_addr="192.168.0.203"

### register_user
Assign the user to a GPU pod and returns its IP address (should be enetered in WebRTC client application)

In [ ]:
%%bash -s "$ws_ip_addr"
curl http://$1:8000/register_user

### status
Retrieve the current status of the assigned pod

In [ ]:
%%bash -s "$ws_ip_addr"
curl http://$1:8000/status

### upload
Upload a *.feature* file for simulation

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/upload -F "file=@/home/jovyan/behave-isaac-bdd/bdd-dsl/examples/generated/var__us-sorting.feature"

### start
Start the simulation on the assigned pod

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/start

### logs_stream
Live stream logs from the simulation

In [ ]:
# start log stream in a terminal in VNC desktop
subprocess.Popen([
                    "gnome-terminal",
                     "--",
                     "bash", "-c", f"curl http://{ws_ip_addr}:8000/logs_stream; exec bash"
                 ])

### start WebRTC client application
In the *Server* field of the aplication, set the pod IP assigned to the user in the **register_user** API call

In [ ]:
# Update the server ip with the ip assigned above after registering
subprocess.Popen([
                    "gnome-terminal",
                     "--",
                     "bash", "-c", f"webrtc-client --no-sandbox; exec bash"
                 ])

### logs?n
Return the last N lines from the simulation logs

In [ ]:
%%bash -s "$ws_ip_addr"
curl http://$1:8000/logs?n=10

### stop
Stops the simulation

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/stop

### release
Deregister the user and frees the assigned pod

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/release

**Note**: At the end of using this virtual lab, please shutdown the JupyterLab *(File->Shut Down)*, as it immediately frees resources for other users